# AL / JWST Shared Utilities Notebook

#### This notebook is intended to be loaded (using %run) by other notebooks in the same directory. It provides common variables and client code for working with the Astrolabe / JWST server via its API.

## An optional helper class to manage interaction with the API:

In [ ]:
import requests
import io

In [ ]:
# default base URL for the AL/JWST API
BASE_URL = 'http://alws.arizona.edu/api'

# Helper class for accessing the ALWS API
class AlwsApi(object):

    def __init__(self, base_url=BASE_URL):
        self.base_url = base_url


    def compose(self, url):
        return f"{self.base_url}/{url}"


    def get(self, url, params=None, **kwargs):
        if (not kwargs):
            kwargs = dict()
        if (not kwargs.get('headers')):
            kwargs['headers'] = dict()
        kwargs['headers']['Authorization'] = self.auth_token
        return requests.get(self.compose(url), params=params, **kwargs)


    def post(self, url, data=None, json=None, **kwargs):
        kwargs['headers']['Authorization'] = self.auth_token
        return requests.post(self.compose(url), data=data, json=json, **kwargs)


    def authorize(self, email, password):
        resp = requests.post(self.compose('auth/token/create/'), data={'email': email, 'password': password})
        if (resp.status_code == 200):
            self.token_pair = resp.json()
            self.auth_token = f"Bearer {self.token_pair.get('access')}"
            self.auth_header = {'Authorization': self.auth_token}
        return resp

In [ ]:
def get_results(resp):
    """
    A successful API call returns a (possibly empty) list of results.
    Extract and return that list.
    """
    assert resp.status_code == 200
    content = resp.json()
    assert content is not None
    return content.get('results')


## Some timing routines, courtesy of Steve Pothiers, NOIR Lab

In [ ]:
import time
def tic():
    tic.start = time.perf_counter()

def toc():
    elapsed_seconds = time.perf_counter() - tic.start
    return elapsed_seconds  # fractional

## Example use of tic-toc:
# tic()
# time.sleep(3)
# print(toc())